# Chicago Weather API

In [35]:
import requests
import pandas as pd

API_KEY = '9aac682673744387b2d27509371709d1'

# chicago coordinates
LAT = 41.8781
LON = -87.6298

# full 2024 months range
months = [
    ('2024-01-01', '2024-02-01'),
    ('2024-02-01', '2024-03-01'),
    ('2024-03-01', '2024-04-01'),
    ('2024-04-01', '2024-05-01'),
    ('2024-05-01', '2024-06-01'),
    ('2024-06-01', '2024-07-01'),
    ('2024-07-01', '2024-08-01'),
    ('2024-08-01', '2024-09-01'),
    ('2024-09-01', '2024-10-01'),
    ('2024-10-01', '2024-11-01'),
    ('2024-11-01', '2024-12-01'),
    ('2024-12-01', '2025-01-01'),
    ('2025-01-01', '2025-02-01'),
    ('2025-02-01', '2025-03-01'),
    ('2025-03-01', '2025-04-01'),
    ('2025-04-01', '2025-05-01'),
    ('2025-05-01', '2025-06-01'),
    ('2025-06-01', '2025-07-01')
]

BASE_URL = 'https://api.weatherbit.io/v2.0/history/hourly'

all_data = []
# fetching data for each month
for start_date, end_date in months:
    print(f"Fetching data for: {start_date} to {end_date}")
    params = {
        'lat': LAT,
        'lon': LON,
        'start_date': start_date,
        'end_date': end_date,
        'key': API_KEY,
        'tz': 'local'
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        json_data = response.json()
        all_data.extend(json_data['data'])
    else:
        print(f"Error fetching {start_date}: {response.status_code} - {response.text}")


chicago_weather = pd.DataFrame(all_data)
print(chicago_weather.head())


Fetching data for: 2024-01-01 to 2024-02-01
Fetching data for: 2024-02-01 to 2024-03-01
Fetching data for: 2024-03-01 to 2024-04-01
Fetching data for: 2024-04-01 to 2024-05-01
Fetching data for: 2024-05-01 to 2024-06-01
Fetching data for: 2024-06-01 to 2024-07-01
Fetching data for: 2024-07-01 to 2024-08-01
Fetching data for: 2024-08-01 to 2024-09-01
Fetching data for: 2024-09-01 to 2024-10-01
Fetching data for: 2024-10-01 to 2024-11-01
Fetching data for: 2024-11-01 to 2024-12-01
Fetching data for: 2024-12-01 to 2025-01-01
Fetching data for: 2025-01-01 to 2025-02-01
Fetching data for: 2025-02-01 to 2025-03-01
Fetching data for: 2025-03-01 to 2025-04-01
Fetching data for: 2025-04-01 to 2025-05-01
Fetching data for: 2025-05-01 to 2025-06-01
Fetching data for: 2025-06-01 to 2025-07-01
   app_temp  azimuth  clouds       datetime  dewpt  dhi  dni  elev_angle  ghi  \
0      -5.7      4.5      87  2024-01-01:06   -1.5    0    0       -71.1    0   
1      -4.9     41.5     100  2024-01-01:07   

In [36]:
chicago_weather['timestamp_local'] = pd.to_datetime(chicago_weather['timestamp_local'])
chicago_weather['date'] = chicago_weather['timestamp_local'].dt.date

# drop dups from overlapping timeframes
chicago_weather = chicago_weather.drop_duplicates(subset='timestamp_local')


unique_dates = chicago_weather['date'].unique()
print(f"Unique dates in the dataset: {len(unique_dates)}")


Unique dates in the dataset: 547


In [40]:
# save df to csv 
chicago_weather.to_csv('chicago_weather.csv', index=False)

# Holiday Data from Python package

In [38]:
import holidays

chi_holidays = pd.DataFrame(chicago_weather['timestamp_local'].dt.date.unique(), columns=['date'])

us_holidays = holidays.US(state='IL', years=[2024, 2025])

chi_holidays['is_holiday'] = chi_holidays['date'].apply(lambda x: x in us_holidays)
chi_holidays['holiday_name'] = chi_holidays['date'].apply(lambda x: us_holidays.get(x) if x in us_holidays else None)

In [39]:
chi_holidays.to_csv('chicago_holidays.csv', index=False)

# Chicago POI Data 

In [ ]:
import osmnx as ox
import geopandas as gpd

place_name = "Chicago, Illinois, USA"
tags = {
    'amenity':   ['restaurant','bar','cafe'],
    'tourism':   ['museum'],
    'leisure':   ['park']
}

pois = ox.features_from_place(place_name, tags) 
pois = pois[pois.geometry.type == 'Point']

pois = pois [['geometry', 'amenity']]

pois.to_csv('CHI_POI.csv', index=False)